In [ ]:
# 🔑 Configure sua API Key da OpenRouter (opcional)
# Se não configurar, o sistema funcionará sem monitoramento LLM

OPENROUTER_API_KEY = ""  # @param {type:"string"}

# Configurar como variável de ambiente se fornecida
import os
if OPENROUTER_API_KEY.strip():
    os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
    print("✅ API Key OpenRouter configurada - Monitor LLM ativado")
else:
    print("⚠️  API Key não configurada - Monitor LLM desabilitado")
    print("   O treinamento funcionará normalmente sem monitoramento inteligente")


In [ ]:
# Verificar GPU disponível
!nvidia-smi

# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navegar para diretório de trabalho
%cd /content


In [ ]:
# Clone do repositório ValeTTS atualizado
!git clone https://github.com/wallaceblaia/ValeTTS-Colab.git ValeTTS
%cd ValeTTS

# Verificar estrutura do projeto
print("📁 Estrutura do projeto:")
!ls -la

# Verificar se os arquivos necessários existem
print("\n🔍 Verificando arquivos essenciais:")
!ls -la requirements.txt setup.py 2>/dev/null || echo "❌ Arquivos não encontrados"
print("📁 Diretório atual:", os.getcwd())


In [ ]:
# Verificar se estamos no diretório correto
import os
print(f"📁 Diretório atual: {os.getcwd()}")

# Verificar se os arquivos existem
if os.path.exists('requirements.txt'):
    print("📦 Instalando dependências do projeto...")
    !pip install -r requirements.txt
else:
    print("⚠️  requirements.txt não encontrado, instalando dependências básicas...")
    !pip install torch torchaudio pytorch-lightning tensorboard pyyaml matplotlib scipy librosa

# Instalar o projeto em modo desenvolvimento (se setup.py existir)
if os.path.exists('setup.py'):
    print("🔧 Instalando projeto em modo desenvolvimento...")
    !pip install -e .
else:
    print("⚠️  setup.py não encontrado, pulando instalação do projeto")
    # Adicionar diretório atual ao PYTHONPATH
    import sys
    if os.getcwd() not in sys.path:
        sys.path.insert(0, os.getcwd())

print("✅ Dependências instaladas com sucesso!")


In [ ]:
# Criar diretório de dados
!mkdir -p data/generated

# Extrair dataset do Google Drive
import os
dataset_path = "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz"

if os.path.exists(dataset_path):
    print("📦 Extraindo dataset...")
    !tar -xzf "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz" -C data/generated/

    # Verificar extração
    print("\n📁 Conteúdo do dataset:")
    !ls -la data/generated/Dataset-Unificado/

    # Verificar metadados
    print("\n📋 Arquivos de metadados:")
    !find data/generated/Dataset-Unificado/ -name "*.json" -o -name "*.csv" | head -5

    print("\n✅ Dataset restaurado com sucesso!")
else:
    print(f"❌ Dataset não encontrado em: {dataset_path}")
    print("   Verifique se o arquivo existe no Google Drive")


In [ ]:
# Verificação completa do sistema
import os

print("🔍 Verificação completa do sistema:")
print(f"📁 Diretório atual: {os.getcwd()}")

# Verificar estrutura essencial
essential_files = [
    'scripts/train_vits2.py',
    'configs/training/vits2_dataset_unificado.yaml',
    'valetts/__init__.py'
]

print("\n📋 Verificando arquivos essenciais:")
for file in essential_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - FALTANDO")

# Verificar se o módulo valetts pode ser importado
try:
    import valetts
    print("✅ Módulo valetts importado com sucesso")
except ImportError as e:
    print(f"⚠️  Erro ao importar valetts: {e}")
    print("   Adicionando diretório ao PYTHONPATH...")
    import sys
    if os.getcwd() not in sys.path:
        sys.path.insert(0, os.getcwd())
    try:
        import valetts
        print("✅ Módulo valetts importado após ajuste do PATH")
    except ImportError as e2:
        print(f"❌ Ainda não conseguiu importar: {e2}")

# Testar script de treinamento
if os.path.exists('scripts/train_vits2.py'):
    print("\n🧪 Testando sistema de treinamento...")
    !python scripts/train_vits2.py --help
    print("\n✅ Sistema funcionando corretamente!")
else:
    print("\n❌ Script de treinamento não encontrado!")
    print("   Listando conteúdo do diretório scripts:")
    !ls -la scripts/ 2>/dev/null || echo "Diretório scripts não encontrado"


In [ ]:
# Mostrar configuração que será usada
import yaml

with open('configs/training/vits2_dataset_unificado.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("📋 Configuração do Treinamento:")
print(f"   • Dataset: {config['data']['dataset_path']}")
print(f"   • Speakers: {config['model']['num_speakers']}")
print(f"   • Epochs: {config['training']['epochs']}")
print(f"   • Batch Size: {config['training']['batch_size']}")
print(f"   • Learning Rate: {config['training']['learning_rate']}")
print(f"   • Mixed Precision: {config['training']['mixed_precision']}")

if 'llm_monitor' in config:
    print(f"   • Monitor LLM: {config['llm_monitor'].get('enabled', False)}")

print("\n🎯 Configuração carregada com sucesso!")


In [ ]:
# 🎯 COMANDO EXATO DO AMBIENTE LOCAL
# python scripts/train_vits2.py --config configs/training/vits2_dataset_unificado.yaml

import os

# Determinar se deve desabilitar LLM baseado na API key
disable_llm = "" if os.environ.get('OPENROUTER_API_KEY') else "--disable-llm"

print("🚀 Iniciando treinamento VITS2...")
print(f"📁 Configuração: configs/training/vits2_dataset_unificado.yaml")
print(f"🔧 Monitor LLM: {'Ativado' if not disable_llm else 'Desabilitado'}")
print("\n" + "="*50)

# Executar treinamento
!python scripts/train_vits2.py --config configs/training/vits2_dataset_unificado.yaml {disable_llm}


In [ ]:
# Carregar TensorBoard para monitoramento
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard/ --port 6006


In [ ]:
# Verificar progresso do treinamento
print("📊 Status do Treinamento:")
!ls -la logs/

print("\n📁 Checkpoints:")
!find logs/ -name "*.ckpt" | tail -5

print("\n📈 Logs mais recentes:")
!tail -10 logs/training.log 2>/dev/null || echo "Log não encontrado ainda"


In [ ]:
# Backup de modelos para Google Drive
import os
from shutil import copytree, ignore_patterns

backup_dir = "/content/drive/MyDrive/ValeTTS-Colab/training_backup"

if os.path.exists("logs/"):
    print("💾 Fazendo backup dos logs e checkpoints...")
    !mkdir -p "{backup_dir}"
    !cp -r logs/ "{backup_dir}/" 2>/dev/null || echo "Backup parcial realizado"
    print(f"✅ Backup salvo em: {backup_dir}")
else:
    print("ℹ️  Nenhum log encontrado para backup")
